<a href="https://colab.research.google.com/github/sw-programmer/Coursework1/blob/main/CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Coursework 1**

1.   ### **Computationally Efficient Eigenfaces**
2.   ### **Incremental PCA**
3.   ### **PCA-LDA for Face Recognition**
4.   ### **K-means codebook**
5.   ### **RF classifier**


**by 20190223	Soohyun Ryu,	20190247	Sangwoo Park**






**1. Setting!**

In [1]:
# import os
# import time
# import torch
# import torch.nn as nn
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt
# import datetime
# import math
# import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
from sklearn.model_selection import train_test_split

Matplotlib is building the font cache; this may take a moment.


In [4]:
mat_file = io.loadmat("./face.mat")
mat_X = mat_file['X']
mat_Y = mat_file['l'][0]

print(mat_file.keys()) 
print(mat_X.shape)                         # 총 52명. 각 10장의 사진.
print(mat_Y.shape) 

dict_keys(['__header__', '__version__', '__globals__', 'X', 'l'])
(2576, 520)
(520,)


In [60]:
df = pd.DataFrame(data=mat_X).transpose()
df['label'] = mat_Y                         # row -> data, col -> feature
print(df)

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values # column 'Target'

X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=35) #sklearn lib
# test/train ratio = 0.2
# random_state -> before dividing, seed for shuffling
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

       0    1    2    3    4    5  ...  2571  2572  2573  2574  2575  label
0    106  103  103  104  101  104  ...    41    41    37    36    41      1
1    130  129  130  129  133  135  ...    43    41    40    36    35      1
2    191  185  174  153  137  114  ...    28    25    22    19    17      1
3    161  156  141  132  133  108  ...    57    58    58    59    60      1
4    161  162  160  151  107  101  ...    55    52    50    47    46      1
..   ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   ...   ...    ...
515  181  170  163  160  156  148  ...    93    75    61    50    45     52
516   60   68   71   77   76   78  ...    86    86    41    48    65     52
517   70   62   83   75   65   71  ...    83    83    87    89    94     52
518  183  250  163   61   74   51  ...   123    86   102    95   106     52
519   77   72   68   64   62   55  ...    73   142    93   103   110     52

[520 rows x 2577 columns]
(416, 2576) (104, 2576) (416,) (104,)


## **1. Computationally Efficient Eignefaces**



In [ ]:
# 평균 얼굴 계산 함수 
def AvgCompute(X): 
  Xbar = np.mean(X, axis=0)  
  matrix_A = X - Xbar 
  return matrix_A, Xbar

# 고유값, 고유벡터 계산 함수
def eig(S): 
  eig_vals, eig_vecs = np.linalg.eig(S) 
  sort_indices = np.argsort(eig_vals)[::-1] # 내림차순 정렬 
  return eig_vals[sort_indices], eig_vecs[:, sort_indices]

# Reconstruction에 쓰일 함수
def reconstruct(X, PC): 
  return (X @ PC) @ PC.T 

# PCA 과정
def PCA(X, num_components): # X == train set, num_components == 선택할 eigen vectoe 개수
  N, D = X.shape 
  matrix_A, X_bar = AvgCompute(X) 
  S = (matrix_A.T @ matrix_A) / N 
  eig_vals, eig_vecs = eig(S) 
  
  principal_vals, principal_components = np.real(eig_vals[:num_components]), np.real(eig_vecs[:,:num_components]) 
  
  reconst_X = reconstruct(matrix_A, principal_components) + X_bar 
  
  return reconst_X, X_bar, principal_vals, principal_components